In [1]:
# 导入模块
%matplotlib inline
import torch
import numpy as np
import pylab as pl
from torch import nn
import re
import torch.nn.functional as F

torch.manual_seed(1)
np.random.seed(1)

In [2]:
# 训练一个基于ERNN神经网络来作诗

## 读入用GloVe处理得到的文字 embeddings，以及句子数据。
import codecs

word_emb_dim = input_size = 1
i2w = {0:''}
w2i = {'':0}

word_emb_dim = 128

with codecs.open('data/word_embeddings_128.txt', mode='r', encoding='utf-8') as f:
    lines = f.readlines()
    n_words = len(lines)+1
    word_embeddings = torch.nn.Embedding(n_words, word_emb_dim)
    for i in range(1, n_words):
        line = lines[i-1].split(' ')
        i2w[i] = line[0]
        w2i[line[0]] = i
        word_embeddings.weight[i] = torch.from_numpy(np.array(line[1:],dtype=np.float32))

max_line_length = 26
poems = []
with codecs.open('data/poems.txt', mode='r', encoding='utf-8') as f:
    for poem in f:
        poem = re.sub('\s','',poem)
        poem = poem.split(':')[-1]
        poem = 'S'+poem+'E'
        if len(poem) < 10 or len(poem) > max_line_length or '(' in poem or u'（' in poem or u'《' in poem or '-' in poem or '_' in poem:
            continue
#        poem = re.split(u'[。；.?？]', poem)
#        for s in poem:
#            if len(s)>3:
#                s += u'。'
        poems.append(map(w2i.get, poem))

n_poems = len(poems)

print( 'Data summary:\n\n number of poems: {}\n number of words: {}\n'.format(n_poems, n_words))
print('Poem examples:\n\n'+'\n'.join([''.join(map(i2w.get, x)) for x in poems[:10]]))
    

Data summary:

 number of poems: 2761
 number of words: 7651

Poem examples:

S四时运灰琯，一夕变冬春。送寒余雪尽，迎岁早梅新。E
S上弦明月半，激箭流星远。落雁带书惊，啼猿映枝转。E
S初秋玉露清，早雁出空鸣。隔云时乱影，因风乍含声。E
S岸曲丝阴聚，波移带影疏。还将眉里翠，来就镜中舒。E
S贞条障曲砌，翠叶贯寒霜。拂牖分龙影，临池待凤翔。E
S散影玉阶柳，含翠隐鸣蝉。微形藏叶里，乱响出风前。E
S盘根直盈渚，交干横倚天。舒华光四海，卷叶荫三川。E
S近谷交萦蕊，遥峰对出莲。径细无全磴，松小未含烟。E
S疾风知劲草，板荡识诚臣。勇夫安识义，智者必怀仁。E
S太液仙舟迥，西园隐上才。未晓征车度，鸡鸣关早开。E


In [16]:
# 定义一个函数，随机返回一个 mini batch，用于训练，由于每一首诗歌的长度不同，我们此处规定每个batch只有一首诗。这样，就可以生成长度可变的诗歌。
def get_batch(batch_size=2):
    batch_raw = [poems[i][:] for i in np.random.randint(0, n_poems, batch_size)]
    max_length = max(map(len, batch_raw))
    for i in range(len(batch_raw)):
        for j in range(len(batch_raw[i]),max_length):
            batch_raw[i].append(0)
    batch_raw = torch.LongTensor(batch_raw).detach().unsqueeze(2).transpose(0,1)
    x = batch_raw[:-1].type(torch.float32)
    y = batch_raw[1:]
    return x, y

def idx2emb(x):
    return word_embeddings(x.type(torch.long)).squeeze(2).detach()
    

# 定义一个函数，输入一个 batch 返回句子
def batch2sent(batch):
    S = []
    batch = batch.type(torch.int32).detach()
    seq_length, batch_size, emb_size = batch.size()
    for i in range(batch_size):
        S.append(''.join(map(i2w.get, batch[:,i,:].view(-1).tolist())))
    return u'\n'.join(S)

x, y = get_batch(1)
print(batch2sent(x))
print(batch2sent(y))

# 定义一个生成器
class Generator(nn.Module):
    def __init__(self, input_size, output_size, hidden_size, n_layers=2, activation=None):
        super(Generator, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.activation = activation
        self.rnn = nn.LSTM(self.input_size, self.hidden_size, num_layers=self.n_layers, dropout=0.01)
        self.output = nn.Linear(self.hidden_size,self.output_size)
        self.softmax = torch.nn.LogSoftmax(dim=-1)
    def init_h(self):
        return (torch.zeros(self.n_layers, self.batch_size, self.hidden_size),torch.zeros(self.n_layers, self.batch_size, self.hidden_size))
    def forward(self, x, h0=None):
        self.seq_length, self.batch_size, self.input_size = x.size()
        if h0 is None:
            h0 = self.init_h()
#            x0 = torch.FloatTensor([w2i['S']]).view(1,1,-1).detach()
#            x0 = idx2emb(x0)
#            y0, h0 = self.rnn(x0,h0)
        y, ht = self.rnn(x,h0)
#        y = torch.cat((y0,y),dim=0)
        y = y.view(-1,self.hidden_size)
        y = self.output(y)
        y = y.view(self.seq_length,self.batch_size,self.output_size)
        y = self.softmax(y)
        return y, ht

def poem_gen(model, w=None):
    with torch.no_grad():
        if not w in w2i or w is None:
            idx = np.random.randint(1,n_words)
            w = i2w[idx]
        else:
            idx = w2i[w]
        ht = None
        x0 = torch.FloatTensor([w2i['S']]).view(1,1,-1).detach()
        x0 = idx2emb(x0)
        y, ht = model(x0, ht)
        x = torch.FloatTensor([w2i[w]]).view(1,1,-1).detach()
        x = idx2emb(x)

        s = ['S']
        s.append(w)
        for t in range(max_line_length):
            y, ht = model(x, ht)
            x = torch.argmax(y, dim=2, keepdim=True)
            w = batch2sent(x)
            s.append(w)
            x = idx2emb(x)
            if w == '':
                break
        return u''.join(s)
    
    
# 训练一个简单的 RNN 模型以生成诗歌

input_size = word_emb_dim
hidden_size = 128
output_size = n_words
activation = torch.relu

model = Generator(input_size, output_size, hidden_size, n_layers=2, activation=activation)


S独酌芳春酒，登楼已半曛。谁惊一行雁，冲断过江云。
独酌芳春酒，登楼已半曛。谁惊一行雁，冲断过江云。E


In [17]:
lr = 1e-3
n_epochs = 10000
last_epoch = -1
disp_interval = 50
batch_size = 1

loss_func = nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=lr)

torch.manual_seed(1)
np.random.seed(1)

def lr_lambda(epoch):
    return 0.99**(epoch/50.0)

scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda, last_epoch=last_epoch)

model.load_state_dict(torch.load('saves/model.pt'))

Loss = []
for epoch in range(n_epochs):
    model.zero_grad()
    x_obs, y_obs = get_batch(batch_size=batch_size)
    x_obs = idx2emb(x_obs)
    y_pred, ht = model(x_obs)
    y1 = torch.argmax(y_pred.detach(),-1,keepdim=True).detach()#[:,:1,:]
    y2 = y_obs.detach()#[:,:1,:]
    y_pred = y_pred.view(-1,output_size)
    y_obs = y_obs.contiguous().view(-1)
    loss = loss_func(y_pred,y_obs)
    loss.backward()
    Loss.append(loss.tolist())
    optimizer.step()
    scheduler.step()
    if epoch % disp_interval == 0:
        print(u'Epoch{}, Loss{}\nPred:\n{}\nObs:\n{}\nRnd:\n{}\n'.format(epoch,loss.tolist(), batch2sent(y1), batch2sent(y2),poem_gen(model)))
        torch.save(model.state_dict(),'saves/model.pt')
window_size = 50
avg_losses = np.array(Loss)[:len(Loss)//50 *50].reshape([-1,window_size]).mean(1)
pl.plot(np.arange(0,len(Loss)//50 *50,window_size), avg_losses,'r-')
pl.xlabel('Time')
pl.ylabel('Loss')
pl.yscale('log')


Epoch0, Loss3.7334856987
Pred:
一得不相人，无中似堪湿。春风如花湿，风人月衣心。E
Obs:
巧笑解迎人，晴雪香堪惜。随风蝶影翻，误点朝衣赤。E
Rnd:
S岁暮春风雨，一家一里西。殷勤E上月，不见去人。E君无得

Epoch50, Loss2.96560144424
Pred:
一有不中下，清泉水不可。何怜谯记内，山水对禅人。E
Obs:
许厕高斋唱，涓泉定不如。可怜谯记室，流水满禅居。E
Rnd:
S觱木不堕，王王韦也。E君君不得，不得相人。E君无人事，

Epoch100, Loss3.53442716599
Pred:
一南千何远，孤风入气里。他时何林里，寒里万边枝。E
Obs:
江旷春潮白，山长晓岫青。他乡临睨极，花柳映边亭。E
Rnd:
S蹀断凭悠悠，秋风入夜深。不怜西上月，偏照不知归。E山月

Epoch150, Loss4.09891700745
Pred:
一门花朝伴，春风意不春。不是一月人，不君不生春。E
Obs:
玉英期共采，云岭独先过。应得灵芝也，诗情一倍多。E
Rnd:
S蓏汩几时夜，孤山月秋深。不知同人事，不见旧时时。E君无

Epoch200, Loss3.83963871002
Pred:
一马三，里，三蝉力无城。何知三上去，不得不人山。E
Obs:
仗剑行千里，微躯感一言。曾为大梁客，不负信陵恩。E
Rnd:
S幻马不出，富心不动。E上有在树，一路始天人。E上无见去

Epoch250, Loss3.07987070084
Pred:
一年无，树，一吟入未归。可怜西郭酒，偏风不家山。E
Obs:
十夜郡城宿，苦吟身未闲。那堪西郭别，雪路问青山。E
Rnd:
S孺马砺，薛不不薛。E南南望，日日半中。E上南南树，南南

Epoch300, Loss3.09251761436
Pred:
三南孟风寂，山声出景声。不知花郭树，日E何悠重。E
Obs:
南楼夜已寂，暗鸟动林间。不见城郭事，沉沉唯四山。E
Rnd:
S襞鶒几无夜，孤中满月明。E君君不得见，不见鹔鹴裘。EE

Epoch350, Loss3.64821505547
Pred:
一年三山远，不君在自迟。可君无年事，不是不中人。E
Obs:
一见嵩山老，吾生恨太迟。问君年几许，曾出上皇时。E
Rnd:
S锱豸自霜尽，龙履真霞尘。一来千里里，一取一家家。E

KeyboardInterrupt: 

In [ ]:
x,y = get_batch(batch_size=2)
print x.size()
print x[:,0,:], y[:,0,:]
print x[:,1,:], y[:,1,:]

In [ ]:
x_pred.requires_grad

In [ ]:
x.size(2)

In [ ]:
x = torch.randn(2,3,4)

In [ ]:
x

In [ ]:
word_embeddings.parameters()

In [ ]:
rnn = nn.LSTM(4, 6, 2)
x = np.random.randn(2, 5, 4)
for i in range(10):
    input = torch.from_numpy(x).type(torch.float32)
    input=input.t()
    rnn.zero_grad()
    output, (hn, cn) = rnn(input)
    print(x.shape)

In [ ]:
torch.transpose(input,0,1).size()

In [6]:
y_obs.size()

torch.Size([25, 5, 1])

In [ ]:
torch.transpose(input,1,0).size()

In [ ]:
torch.topk(torch.randn(1,1,100),1,-1)[1].shape

In [ ]:
input

In [ ]:
x.size()

In [ ]:
a[:]

In [ ]:
x.size()

In [ ]:
y.size()